# Model Evaluation (Weighted) – Notebook Guide

This notebook evaluates models with class/observation weights applied.

## What this notebook does
- Compute weighted metrics (e.g., weighted AUC, threshold metrics)
- Plot diagnostic figures considering weights
- Summarize results per model/run and export

## Inputs
- Predictions/scores, ground-truth labels, and weights per observation
- Optional: CV fold info or test set indicators

## Workflow
1. Load predictions, labels, and weights
2. Validate alignment and handle missing values
3. Compute weighted metrics across thresholds/folds
4. Plot weighted ROC/curves and summaries
5. Save metrics tables and figures

## Outputs
- Weighted per-model/per-fold metrics tables
- Plots reflecting weights
- CSV/JSON exports for downstream use

## Notes
- Ensure weights are normalized or in intended scale
- Use consistent preprocessing as training
- Fix random seeds for reproducibility where applicable


# Notebook Overview

This notebook evaluates weighted SDMs with metrics and plots, mirroring standard evaluation but accounting for weights in analysis where relevant.

- Key steps: load weighted predictions, compute metrics, plot curves, thresholds, reporting
- Inputs: weighted model predictions and labels
- Outputs: evaluation tables and plots
- Run order: After weighted model training.


# Weighted MaxEnt Model Evaluation and Performance Assessment

This notebook provides comprehensive evaluation of **weighted MaxEnt species distribution models**, focusing on performance assessment that accounts for sample weights and data quality differences. Unlike standard model evaluation, this version incorporates **weighted metrics** to properly assess model performance when training data has been weighted.

## Key Features of Weighted Model Evaluation:

### 1. **Weighted Performance Metrics**:
- **Weighted AUC**: Area Under ROC Curve accounting for sample weights
- **Weighted PR-AUC**: Precision-Recall AUC with weight integration
- **Weighted Sensitivity/Specificity**: Performance metrics adjusted for data quality
- **Weighted Precision/Recall**: Classification metrics incorporating sample weights

### 2. **Advanced Evaluation Approaches**:
- **Cross-Validation**: K-fold validation with weighted samples
- **Spatial Validation**: Geographic partitioning with weight consideration
- **Temporal Validation**: Time-based splits accounting for temporal weights
- **Bootstrap Validation**: Resampling with weight preservation

### 3. **Bias Assessment**:
- **Spatial Bias Analysis**: Evaluate model performance across different regions
- **Temporal Bias Assessment**: Performance across different time periods
- **Source Bias Evaluation**: Performance across different data sources
- **Quality Bias Analysis**: Performance across different data quality levels

## Applications:
- **Model Validation**: Comprehensive assessment of weighted model performance
- **Bias Detection**: Identify remaining biases after weighting
- **Performance Comparison**: Compare weighted vs. unweighted models
- **Quality Control**: Validate that weighting improves model reliability

In [1]:
############### WEIGHTED MODEL EVALUATION CONFIGURATION - MODIFY AS NEEDED ###############

# Species and region settings for weighted model evaluation
#specie = 'leptocybe-invasa'  # Target species: 'leptocybe-invasa' or 'thaumastocoris-peregrinus'
#pseudoabsence = 'random'  # Background point strategy: 'random', 'biased', 'biased-land-cover'
#training = 'east-asia'  # Training region: 'sea', 'australia', 'east-asia', etc.
#interest = 'south-east-asia'  # Test region: can be same as training or different
#savefig = True  # Save generated evaluation plots and metrics

# Environmental variable configuration
bio = bio1  # Bioclimatic variable identifier

# Evaluation settings (specific to weighted model evaluation)
# evaluation_method = 'cross_validation'  # 'cross_validation', 'spatial_validation', 'temporal_validation'
# n_folds = 5  # Number of folds for cross-validation
# spatial_buffer = 100  # Buffer distance (km) for spatial validation
# temporal_split = 0.7  # Proportion of data for training in temporal validation

# Weighted metrics configuration
# include_weighted_metrics = True  # Calculate weighted performance metrics
# include_unweighted_metrics = True  # Calculate standard metrics for comparison
# weight_threshold = 0.1  # Minimum weight threshold for sample inclusion

###########################################################

NameError: name 'bio1' is not defined

In [ ]:
# =============================================================================
# IMPORT REQUIRED LIBRARIES
# =============================================================================

import os  # File system operations

import numpy as np  # Numerical computing
import xarray as xr  # Multi-dimensional labeled arrays (raster data)
import pandas as pd  # Data manipulation and analysis
import geopandas as gpd  # Geospatial data handling

import elapid as ela  # Species distribution modeling library

from shapely import wkt  # Well-Known Text (WKT) geometry parsing
from elapid import utils  # Utility functions for elapid
from sklearn import metrics, inspection  # Machine learning metrics and model inspection

import matplotlib.pyplot as plt  # Plotting and visualization

import warnings
warnings.filterwarnings("ignore")  # Suppress warning messages for cleaner output

# Configure matplotlib for publication-quality plots
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [ ]:
def subplot_layout(nplots):
    """
    Calculate optimal subplot layout for given number of plots
    
    Parameters:
    -----------
    nplots : int
        Number of plots to arrange
    
    Returns:
    --------
    ncols, nrows : tuple
        Number of columns and rows for subplot layout
    """
    
    # Calculate square root and round up for balanced layout
    ncols = min(int(np.ceil(np.sqrt(nplots))), 4)  # Max 4 columns
    nrows = int(np.ceil(nplots / ncols))  # Calculate rows needed
    
    return ncols, nrows

In [ ]:
# =============================================================================
# SET UP FILE PATHS
# =============================================================================
# Define directory structure for organizing weighted model evaluation outputs

docs_path = os.path.join(os.path.dirname(os.getcwd()), 'docs')  # Documentation directory
out_path = os.path.join(os.path.dirname(os.getcwd()), 'out', specie)  # Species-specific output directory
figs_path = os.path.join(os.path.dirname(os.getcwd()), 'figs')  # Figures directory
output_path = os.path.join(out_path, 'output')  # Model output directory

## 1. Weighted Training Model Performance Assessment

This section evaluates the performance of the weighted MaxEnt model on the training data. Key aspects include:

### **Weighted vs. Unweighted Metrics**:
- **Standard Metrics**: Traditional AUC, PR-AUC, sensitivity, specificity
- **Weighted Metrics**: Performance metrics accounting for sample weights
- **Comparison Analysis**: Evaluate improvement from weighting approach

### **Performance Indicators**:
- **ROC-AUC**: Area Under Receiver Operating Characteristic curve
- **PR-AUC**: Area Under Precision-Recall curve (important for imbalanced data)
- **Sensitivity**: True Positive Rate (ability to detect presences)
- **Specificity**: True Negative Rate (ability to detect absences)
- **Precision**: Positive Predictive Value
- **F1-Score**: Harmonic mean of precision and recall

### **Weighted Evaluation Benefits**:
- **Quality-Aware Assessment**: Metrics reflect data quality differences
- **Bias-Corrected Performance**: Reduced influence of low-quality samples
- **Robust Validation**: More reliable performance estimates

## References for Species Distribution Model Evaluation

### **Model Output Interpretation**:
- [SDM Model Outputs Interpretation](https://support.ecocommons.org.au/support/solutions/articles/6000256107-interpretation-of-sdm-model-outputs)
- [Presence-Only Prediction in GIS](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/how-presence-only-prediction-works.htm)
- [MaxEnt 101: Species Distribution Modeling](https://www.esri.com/arcgis-blog/products/arcgis-pro/analytics/presence-only-prediction-maxent-101-using-gis-to-model-species-distribution/)

### **Performance Metrics**:
- [ROC Curves Demystified](https://towardsdatascience.com/receiver-operating-characteristic-curves-demystified-in-python-bd531a4364d0)
- [Precision-Recall AUC Guide](https://www.aporia.com/learn/ultimate-guide-to-precision-recall-auc-understanding-calculating-using-pr-auc-in-ml/)
- [F1-Score, Accuracy, ROC-AUC, and PR-AUC Metrics](https://deepchecks.com/f1-score-accuracy-roc-auc-and-pr-auc-metrics-for-models/)

### **Weighted Model Evaluation**:
- **Sample Weighting**: How to properly evaluate models trained with sample weights
- **Bias Correction**: Assessing the effectiveness of weighting strategies
- **Quality Integration**: Incorporating data quality into performance assessment

In [ ]:
# =============================================================================
# LOAD WEIGHTED MODEL AND TRAINING DATA
# =============================================================================
# Load the trained weighted MaxEnt model and associated training data for evaluation

# Build experiment directory name (keeps runs organized by config)
# Alternate naming (older): 'exp_%s_%s_%s' % (pseudoabsence, training, interest)
experiment_name = 'exp_%s_%s_%s_%s_%s' % (model_prefix, pseudoabsence, training, topo, ndvi)
exp_path = os.path.join(output_path, experiment_name)  # Path to experiment directory

# Construct expected filenames produced during training for this run
train_input_data_name = '%s_model-train_input-data_%s_%s_%s_%s_%s.csv' % (model_prefix, specie, pseudoabsence, training, bio, iteration)
run_name = '%s_model-train_%s_%s_%s_%s_%s.ela' % (model_prefix, specie, pseudoabsence, training, bio, iteration)
nc_name = '%s_model-train_%s_%s_%s_%s_%s.nc' % (model_prefix, specie, pseudoabsence, training, bio, iteration)

In [ ]:
# =============================================================================
# LOAD TRAINING DATA WITH SAMPLE WEIGHTS
# =============================================================================
# Load training data including sample weights for weighted model evaluation

# Load training data from CSV file (index_col=0 to drop old index column)
df = pd.read_csv(os.path.join(exp_path, train_input_data_name), index_col=0)
# Parse WKT strings into shapely geometries
df['geometry'] = df['geometry'].apply(wkt.loads)
# Wrap as GeoDataFrame with WGS84 CRS
train = gpd.GeoDataFrame(df, crs='EPSG:4326')

# Split predictors/labels/weights for weighted evaluation
x_train = train.drop(columns=['class', 'SampleWeight', 'geometry'])  # Environmental variables only
y_train = train['class']  # Presence/absence labels (0/1)
sample_weight_train = train['SampleWeight']  # Sample weights aligned with rows

# Load fitted weighted MaxEnt model
model_train = utils.load_object(os.path.join(exp_path, run_name))

# Predict probabilities on training set (for curves/metrics)
y_train_predict = model_train.predict(x_train)
# Optional: impute NaN probabilities to 0.5 (neutral)
# y_train_predict = np.nan_to_num(y_train_predict, nan=0.5)

In [ ]:
# Model training performance metrics

# ROC curve and AUC (unweighted vs weighted)
# fpr/tpr are computed from predicted probabilities; weights adjust contribution per sample
fpr_train, tpr_train, thresholds = metrics.roc_curve(y_train, y_train_predict)
auc_train = metrics.roc_auc_score(y_train, y_train_predict)
auc_train_weighted = metrics.roc_auc_score(y_train, y_train_predict, sample_weight=sample_weight_train)

# Precision-Recall curve and PR-AUC (more informative on class imbalance)
precision_train, recall_train, _ = metrics.precision_recall_curve(y_train, y_train_predict)
pr_auc_train = metrics.auc(recall_train, precision_train)
# Weighted PR curve uses sample weights to compute precision/recall
precision_train_w, recall_train_w, _ = metrics.precision_recall_curve(y_train, y_train_predict, sample_weight=sample_weight_train)
pr_auc_train_weighted = metrics.auc(recall_train_w, precision_train_w)

# Report metrics
print(f"Training ROC-AUC score: {auc_train:0.3f}")
print(f"Training ROC-AUC Weighted score  : {auc_train_weighted:0.3f}")
print(f"PR-AUC Score: {pr_auc_train:0.3f}")
print(f"PR-AUC Weighted Score: {pr_auc_train_weighted:0.3f}")

|  |  | Specie existance |  |
| ------ | :-------: | :------: | :-------: |
| |  | **+** | **--** |
| **Specie observed** | **+** | True Positive (TP) | False Positive (FP) |
| | **--** | False Negative (FN) | True Negative (TN) |
| | | **All existing species (TP + FN)** | **All non-existing species (FP + TN)** |


$$TPR = \frac{TP}{TP + FN}$$
$$FPR = \frac{FP}{FP + TN}$$

In [ ]:
# Visualize training distributions and curves
fig, ax = plt.subplots(ncols=3, figsize=(18, 6), constrained_layout=True)

# Left: Predicted probability distributions for presence vs pseudo-absence
ax[0].hist(y_train_predict[y_train == 0], bins=np.linspace(0, 1, int((y_train == 0).sum() / 100 + 1)),
           density=True, color='tab:red', alpha=0.7, label='pseudo-absence')
ax[0].hist(y_train_predict[y_train == 1], bins=np.linspace(0, 1, int((y_train == 1).sum() / 10 + 1)),
           density=True, color='tab:green', alpha=0.7, label='presence')
ax[0].set_xlabel('Relative Occurrence Probability')
ax[0].set_ylabel('Counts')
ax[0].set_title('Probability Distribution')
ax[0].legend(loc='upper right')

# Middle: ROC curve (random vs perfect baselines + model)
ax[1].plot([0, 1], [0, 1], '--', label='AUC score: 0.5 (No Skill)', color='gray')
ax[1].text(0.4, 0.4, 'random classifier', fontsize=12, color='gray', rotation=45, rotation_mode='anchor',
           horizontalalignment='left', verticalalignment='bottom', transform=ax[1].transAxes)
ax[1].plot([0, 0, 1], [0, 1, 1], '--', label='AUC score: 1 (Ideal Model)', color='tab:blue', zorder=-1)
ax[1].text(0, 1, '  perfect classifier', fontsize=12, color='tab:blue', horizontalalignment='left', verticalalignment='bottom')
ax[1].scatter(0, 1, marker='*', s=100, color='tab:blue')
# Overlay model ROC (unweighted and weighted AUC labels)
ax[1].plot(fpr_train, tpr_train, label=f'AUC score: {auc_train:0.3f}', color='tab:orange')
ax[1].plot(fpr_train, tpr_train, label=f'AUC Weighted score: {auc_train_weighted:0.3f}', color='tab:cyan', linestyle='-.')
ax[1].axis('equal')
ax[1].set_xlabel('False Positive Rate')
ax[1].set_ylabel('True Positive Rate')
ax[1].set_title('MaxEnt ROC Curve')
ax[1].legend(loc='lower right')

# Right: Precision-Recall curve (random/perfect baselines + model)
ax[2].plot([0, 1], [0.5, 0.5], '--', color='gray', label='AUC score: 0.5 (No Skill)')
ax[2].text(0.5, 0.52, 'random classifier', fontsize=12, color='gray', horizontalalignment='center', verticalalignment='center')
ax[2].plot([0, 1, 1], [1, 1, 0], '--', label='AUC score: 1 (Ideal Model)', color='tab:blue', zorder=-1)
ax[2].text(1, 1, 'perfect classifier  ', fontsize=12, color='tab:blue', horizontalalignment='right', verticalalignment='bottom')
ax[2].scatter(1, 1, marker='*', s=100, color='tab:blue')
# Overlay model PR curves (unweighted and weighted AUC labels)
ax[2].plot(recall_train, precision_train, label=f'AUC score: {pr_auc_train:0.3f}', color='tab:orange')
ax[2].plot(recall_train_w, precision_train_w, label=f"AUC Weighted score: {pr_auc_train_weighted:0.3f}", color='tab:cyan', linestyle='-.')
ax[2].axis('equal')
ax[2].set_xlabel('Recall')
ax[2].set_ylabel('Precision')
ax[2].set_title('MaxEnt PR Curve')
ax[2].legend(loc='lower left')

In [ ]:
# Save figures if requested. Uses different filename patterns for current vs future scenarios.
# Note: 'models' is used to gate inclusion of model prefix; ensure it exists in your session.
if savefig:
    if Future:
        if models:  # include model identifier when available
            file_path = os.path.join(
                figs_path,
                '06_roc-pr-auc_%s_%s_%s_%s_%s_future.png' % (specie, training, bio, model_prefix, iteration),
            )
        else:
            file_path = os.path.join(
                figs_path,
                '06_roc-pr-auc_%s_%s_%s_%s_future.png' % (specie, training, bio, iteration),
            )
        fig.savefig(file_path, transparent=True, bbox_inches='tight')

    else:
        if models:
            file_path = os.path.join(
                figs_path,
                '06_roc-pr-auc_%s_%s_%s_%s_%s.png' % (specie, training, bio, model_prefix, iteration),
            )
        else:
            # Fallback: omit model prefix when not specified
            file_path = os.path.join(
                figs_path,
                '06_roc-pr-auc_%s_%s_%s_%s.png' % (specie, training, bio, iteration),
            )
        fig.savefig(file_path, transparent=True, bbox_inches='tight')


## 2. Test model performance

In [ ]:
test_input_data_name = '%s_model-test_input-data_%s_%s_%s_%s_%s.csv' %(model_prefix, specie, pseudoabsence, interest, bio, iteration)

In [ ]:
# Load held-out test dataset for evaluation
# Note: index_col=0 drops the old index saved during export
df = pd.read_csv(os.path.join(exp_path, test_input_data_name), index_col=0)
# Convert WKT geometry back to shapely objects
df['geometry'] = df['geometry'].apply(wkt.loads)
# Wrap as GeoDataFrame (WGS84 CRS)
test = gpd.GeoDataFrame(df, crs='EPSG:4326')

In [ ]:
# Split predictors/labels/weights for test set
x_test = test.drop(columns=['class', 'SampleWeight', 'geometry'])
y_test = test['class']
sample_weight_test = test['SampleWeight']

# Predict probabilities on the test set using the trained model
y_test_predict = model_train.predict(x_test)
# Optional: impute NaN probabilities to 0.5 if present
# y_test_predict = np.nan_to_num(y_test_predict, nan=0.5)

In [ ]:
# Test set metrics: ROC/PR curves and AUCs (unweighted vs weighted)
# ROC
fpr_test, tpr_test, _ = metrics.roc_curve(y_test, y_test_predict)
auc_test = metrics.roc_auc_score(y_test, y_test_predict)
auc_test_weighted = metrics.roc_auc_score(y_test, y_test_predict, sample_weight=sample_weight_test)

# Precision-Recall (PR)
precision_test, recall_test, _ = metrics.precision_recall_curve(y_test, y_test_predict)
pr_auc_test = metrics.auc(recall_test, precision_test)
precision_test_w, recall_test_w, _ = metrics.precision_recall_curve(y_test, y_test_predict, sample_weight=sample_weight_test)
pr_auc_test_weighted = metrics.auc(recall_test_w, precision_test_w)

# Print summary of training vs test for quick comparison
print(f"Training ROC-AUC score: {auc_train:0.3f}")
print(f"Training ROC-AUC Weighted score: {auc_train_weighted:0.3f}")
print(f"Test ROC-AUC score: {auc_test:0.3f}")
print(f"Test ROC-AUC Weighted score: {auc_test_weighted:0.3f}")

print(f"Training PR-AUC Score: {pr_auc_train:0.3f}")
print(f"Training PR-AUC Weighted Score: {pr_auc_train_weighted:0.3f}")
print(f"Test PR-AUC Score: {pr_auc_test:0.3f}")
print(f"Test PR-AUC Weighted Score: {pr_auc_test_weighted:0.3f}")

In [ ]:
# Visualize test distributions and curves alongside training for comparison
fig, ax = plt.subplots(ncols=3, figsize=(18, 6), constrained_layout=True)

# Left: Predicted probability distributions on test set
ax[0].hist(y_test_predict[y_test == 0], bins=np.linspace(0, 1, int((y_test == 0).sum() / 100 + 1)),
           density=True, color='tab:red', alpha=0.7, label='pseudo-absence')
ax[0].hist(y_test_predict[y_test == 1], bins=np.linspace(0, 1, int((y_test == 1).sum() / 10 + 1)),
           density=True, color='tab:green', alpha=0.7, label='presence')
ax[0].set_xlabel('Relative Occurrence Probability')
ax[0].set_ylabel('Counts')
ax[0].set_title('Probability Distribution')
ax[0].legend(loc='upper right')

# Middle: ROC curves (train vs test, with weighted variants labeled)
ax[1].plot([0, 1], [0, 1], '--', label='AUC score: 0.5 (No Skill)', color='gray')
ax[1].text(0.4, 0.4, 'random classifier', fontsize=12, color='gray', rotation=45, rotation_mode='anchor',
           horizontalalignment='left', verticalalignment='bottom', transform=ax[1].transAxes)
ax[1].plot([0, 0, 1], [0, 1, 1], '--', label='AUC score: 1 (Ideal Model)', color='tab:blue', zorder=-1)
ax[1].text(0, 1, '  perfect classifier', fontsize=12, color='tab:blue', horizontalalignment='left', verticalalignment='bottom')
ax[1].scatter(0, 1, marker='*', s=100, color='tab:blue')
ax[1].plot(fpr_train, tpr_train, label=f'AUC train score: {auc_train:0.3f}', color='tab:orange')
ax[1].plot(fpr_train, tpr_train, label=f'AUC Weighted train score: {auc_train_weighted:0.3f}', color='tab:cyan', linestyle='-.')
ax[1].plot(fpr_test, tpr_test, label=f'AUC test score: {auc_test:0.3f}', color='tab:green')
ax[1].plot(fpr_test, tpr_test, label=f'AUC Weighted test score: {auc_test_weighted:0.3f}', color='tab:olive', linestyle='-.')
ax[1].axis('equal')
ax[1].set_xlabel('False Positive Rate')
ax[1].set_ylabel('True Positive Rate')
ax[1].set_title('MaxEnt ROC Curve')
ax[1].legend(loc='lower right')

# Right: PR curves (train vs test)
ax[2].plot([0, 1], [0.5, 0.5], '--', color='gray', label='AUC score: 0.5 (No Skill)')
ax[2].text(0.5, 0.52, 'random classifier', fontsize=12, color='gray', horizontalalignment='center', verticalalignment='center')
ax[2].plot([0, 1, 1], [1, 1, 0], '--', label='AUC score: 1 (Ideal Model)', color='tab:blue', zorder=-1)
ax[2].text(1, 1, 'perfect classifier  ', fontsize=12, color='tab:blue', horizontalalignment='right', verticalalignment='bottom')
ax[2].scatter(1, 1, marker='*', s=100, color='tab:blue')
ax[2].plot(recall_train, precision_train, label=f'AUC train score: {pr_auc_train:0.3f}', color='tab:orange')
ax[2].plot(recall_train_w, precision_train_w, label=f"AUC train Weighted score: {pr_auc_train_weighted:0.3f}", color='tab:cyan', linestyle='-.')
ax[2].plot(recall_test, precision_test, label=f'AUC test score: {pr_auc_test:0.3f}', color='tab:green')
ax[2].plot(recall_test_w, precision_test_w, label=f'AUC test Weighted score: {pr_auc_test_weighted:0.3f}', color='tab:olive', linestyle='-.')
ax[2].axis('equal')
ax[2].set_xlabel('Recall')
ax[2].set_ylabel('Precision')
ax[2].set_title('MaxEnt PR Curve')
ax[2].legend(loc='lower left')

In [ ]:
# Save test figures if requested (future vs current naming handled similarly to training)
if savefig:
    if Future:
        if models:
            file_path = os.path.join(
                figs_path,
                '06_roc-pr-auc_%s_%s_%s_%s_%s_future.png' % (specie, interest, bio, model_prefix, iteration),
            )
        else:
            file_path = os.path.join(
                figs_path,
                '06_roc-pr-auc_%s_%s_%s_%s_future.png' % (specie, interest, bio, iteration),
            )
        fig.savefig(file_path, transparent=True, bbox_inches='tight')

    else:
        if model_prefix:
            file_path = os.path.join(
                figs_path,
                '06_roc-pr-auc_%s_%s_%s_%s_%s.png' % (specie, interest, bio, model_prefix, iteration),
            )
        else:
            file_path = os.path.join(
                figs_path,
                '06_roc-pr-auc_%s_%s_%s_%s.png' % (specie, interest, bio, iteration),
            )
        fig.savefig(file_path, transparent=True, bbox_inches='tight')

## 3. Evaluate model

### 3.2 Partial dependence plot/ Response curves

In [ ]:
# fig, ax = model_train.partial_dependence_plot(x, labels=labels, dpi=100, n_bins=30)

## 4. Comprehensive Variable Importance Analysis

This section performs a thorough analysis of variable importance by:

1. **Initial Analysis**: Running the model with all 19 bioclimatic variables to establish baseline importance
2. **Iterative Removal**: Systematically removing the least important variables until we reach ~5 most important variables
3. **Performance Tracking**: Monitoring model performance as variables are removed
4. **Final Recommendations**: Identifying the optimal subset of variables for the species distribution model

### Methodology:
- **Permutation Importance**: Measures the drop in model performance when each variable is randomly shuffled
- **Iterative Backward Elimination**: Removes least important variables one at a time
- **Performance Monitoring**: Tracks AUC, PR-AUC, and other metrics throughout the process
- **Cross-Validation**: Ensures robust importance estimates


In [ ]:
# =============================================================================
# COMPREHENSIVE VARIABLE IMPORTANCE ANALYSIS
# =============================================================================

import time
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Initialize storage for results
importance_results = {}
performance_history = {}
variable_subsets = {}

# Get current variable names from training data
current_variables = list(x_train.columns)
print(f"Starting with {len(current_variables)} variables:")
print(f"Variables: {current_variables}")

# Store initial performance metrics
initial_metrics = {
    'train_auc': auc_train,
    'train_auc_weighted': auc_train_weighted,
    'train_pr_auc': pr_auc_train,
    'train_pr_auc_weighted': pr_auc_train_weighted,
    'test_auc': auc_test,
    'test_auc_weighted': auc_test_weighted,
    'test_pr_auc': pr_auc_test,
    'test_pr_auc_weighted': pr_auc_test_weighted
}

performance_history['all_variables'] = initial_metrics
variable_subsets['all_variables'] = current_variables.copy()

print(f"\nInitial Performance (All {len(current_variables)} variables):")
print(f"Training AUC: {auc_train:.3f} (weighted: {auc_train_weighted:.3f})")
print(f"Training PR-AUC: {pr_auc_train:.3f} (weighted: {pr_auc_train_weighted:.3f})")
print(f"Test AUC: {auc_test:.3f} (weighted: {auc_test_weighted:.3f})")
print(f"Test PR-AUC: {pr_auc_test:.3f} (weighted: {pr_auc_test_weighted:.3f})")


In [ ]:
# =============================================================================
# ITERATIVE VARIABLE REMOVAL FUNCTION
# =============================================================================

def iterative_variable_removal(x_train, y_train, sample_weight_train, x_test, y_test, sample_weight_test, 
                              target_variables=5, min_variables=3):
    """
    Iteratively remove least important variables until reaching target number.
    
    Parameters:
    -----------
    x_train, y_train, sample_weight_train : training data
    x_test, y_test, sample_weight_test : test data  
    target_variables : int, target number of variables to keep
    min_variables : int, minimum number of variables to keep
    
    Returns:
    --------
    results : dict, containing importance rankings and performance history
    """
    
    results = {
        'importance_rankings': {},
        'performance_history': {},
        'removed_variables': [],
        'final_variables': []
    }
    
    current_x_train = x_train.copy()
    current_x_test = x_test.copy()
    current_vars = list(current_x_train.columns)
    iteration = 0
    
    print(f"Starting iterative removal from {len(current_vars)} to {target_variables} variables...")
    
    while len(current_vars) > max(target_variables, min_variables):
        iteration += 1
        print(f"\n--- Iteration {iteration}: {len(current_vars)} variables remaining ---")
        
        # Train model with current variables
        model_iter = ela.MaxentModel()
        model_iter.fit(current_x_train, y_train, sample_weight=sample_weight_train)
        
        # Calculate permutation importance
        pi = inspection.permutation_importance(
            model_iter, current_x_train, y_train, 
            sample_weight=sample_weight_train, n_repeats=10
        )
        
        # Get importance scores and rank variables
        importance_scores = pi.importances.mean(axis=1)
        var_importance = dict(zip(current_vars, importance_scores))
        sorted_vars = sorted(var_importance.items(), key=lambda x: x[1], reverse=True)
        
        # Store ranking for this iteration
        results['importance_rankings'][f'iteration_{iteration}'] = {
            'variables': current_vars.copy(),
            'importance_scores': var_importance.copy(),
            'sorted_ranking': sorted_vars.copy()
        }
        
        # Calculate performance metrics
        y_train_pred = model_iter.predict(current_x_train)
        y_test_pred = model_iter.predict(current_x_test)
        
        # Training metrics
        train_auc = metrics.roc_auc_score(y_train, y_train_pred)
        train_auc_weighted = metrics.roc_auc_score(y_train, y_train_pred, sample_weight=sample_weight_train)
        train_precision, train_recall, _ = metrics.precision_recall_curve(y_train, y_train_pred)
        train_pr_auc = metrics.auc(train_recall, train_precision)
        train_precision_w, train_recall_w, _ = metrics.precision_recall_curve(y_train, y_train_pred, sample_weight=sample_weight_train)
        train_pr_auc_weighted = metrics.auc(train_recall_w, train_precision_w)
        
        # Test metrics
        test_auc = metrics.roc_auc_score(y_test, y_test_pred)
        test_auc_weighted = metrics.roc_auc_score(y_test, y_test_pred, sample_weight=sample_weight_test)
        test_precision, test_recall, _ = metrics.precision_recall_curve(y_test, y_test_pred)
        test_pr_auc = metrics.auc(test_recall, test_precision)
        test_precision_w, test_recall_w, _ = metrics.precision_recall_curve(y_test, y_test_pred, sample_weight=sample_weight_test)
        test_pr_auc_weighted = metrics.auc(test_recall_w, test_precision_w)
        
        # Store performance
        results['performance_history'][f'iteration_{iteration}'] = {
            'n_variables': len(current_vars),
            'train_auc': train_auc,
            'train_auc_weighted': train_auc_weighted,
            'train_pr_auc': train_pr_auc,
            'train_pr_auc_weighted': train_pr_auc_weighted,
            'test_auc': test_auc,
            'test_auc_weighted': test_auc_weighted,
            'test_pr_auc': test_pr_auc,
            'test_pr_auc_weighted': test_pr_auc_weighted
        }
        
        # Print current performance
        print(f"Performance with {len(current_vars)} variables:")
        print(f"  Train AUC: {train_auc:.3f} (weighted: {train_auc_weighted:.3f})")
        print(f"  Test AUC: {test_auc:.3f} (weighted: {test_auc_weighted:.3f})")
        print(f"  Train PR-AUC: {train_pr_auc:.3f} (weighted: {train_pr_auc_weighted:.3f})")
        print(f"  Test PR-AUC: {test_pr_auc:.3f} (weighted: {test_pr_auc_weighted:.3f})")
        
        # Identify least important variable
        least_important_var = sorted_vars[-1][0]
        least_important_score = sorted_vars[-1][1]
        
        print(f"Least important variable: {least_important_var} (importance: {least_important_score:.4f})")
        
        # Remove least important variable
        current_x_train = current_x_train.drop(columns=[least_important_var])
        current_x_test = current_x_test.drop(columns=[least_important_var])
        current_vars.remove(least_important_var)
        results['removed_variables'].append(least_important_var)
        
        print(f"Removed {least_important_var}. Variables remaining: {current_vars}")
    
    results['final_variables'] = current_vars.copy()
    print(f"\nFinal variable set ({len(current_vars)} variables): {current_vars}")
    
    return results


In [ ]:
# =============================================================================
# RUN ITERATIVE VARIABLE REMOVAL ANALYSIS
# =============================================================================

print("="*80)
print("COMPREHENSIVE VARIABLE IMPORTANCE ANALYSIS")
print("="*80)

# Run the iterative removal process
start_time = time.time()

# Set target to 5 variables (can be adjusted)
target_vars = 5
min_vars = 3

# Run iterative removal
removal_results = iterative_variable_removal(
    x_train, y_train, sample_weight_train,
    x_test, y_test, sample_weight_test,
    target_variables=target_vars,
    min_variables=min_vars
)

end_time = time.time()
print(f"\nAnalysis completed in {end_time - start_time:.1f} seconds")

# Store results for later analysis
importance_results['iterative_removal'] = removal_results


In [ ]:
# =============================================================================
# ANALYZE AND VISUALIZE RESULTS
# =============================================================================

# Extract performance trends
iterations = list(removal_results['performance_history'].keys())
n_vars = [removal_results['performance_history'][iter]['n_variables'] for iter in iterations]
train_aucs = [removal_results['performance_history'][iter]['train_auc'] for iter in iterations]
test_aucs = [removal_results['performance_history'][iter]['test_auc'] for iter in iterations]
train_aucs_weighted = [removal_results['performance_history'][iter]['train_auc_weighted'] for iter in iterations]
test_aucs_weighted = [removal_results['performance_history'][iter]['test_auc_weighted'] for iter in iterations]

# Add initial performance (all variables)
n_vars.insert(0, len(x_train.columns))
train_aucs.insert(0, auc_train)
test_aucs.insert(0, auc_test)
train_aucs_weighted.insert(0, auc_train_weighted)
test_aucs_weighted.insert(0, auc_test_weighted)

print("Performance Summary:")
print("="*50)
print(f"{'Variables':<12} {'Train AUC':<10} {'Test AUC':<10} {'Train AUC-W':<12} {'Test AUC-W':<12}")
print("-"*60)
for i, n_var in enumerate(n_vars):
    print(f"{n_var:<12} {train_aucs[i]:<10.3f} {test_aucs[i]:<10.3f} {train_aucs_weighted[i]:<12.3f} {test_aucs_weighted[i]:<12.3f}")

# Get final variable ranking
final_iteration = f"iteration_{len(iterations)}"
final_ranking = removal_results['importance_rankings'][final_iteration]['sorted_ranking']

print(f"\nFinal Variable Ranking (Top {len(removal_results['final_variables'])} variables):")
print("="*60)
for i, (var, importance) in enumerate(final_ranking, 1):
    print(f"{i:2d}. {var:<15} (importance: {importance:.4f})")

print(f"\nRemoved Variables (in order of removal):")
print("="*40)
for i, var in enumerate(removal_results['removed_variables'], 1):
    print(f"{i:2d}. {var}")


In [ ]:
# =============================================================================
# CREATE COMPREHENSIVE VISUALIZATION
# =============================================================================

# Create a comprehensive figure showing the analysis results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comprehensive Variable Importance Analysis', fontsize=16, fontweight='bold')

# 1. Performance vs Number of Variables
ax1 = axes[0, 0]
ax1.plot(n_vars, train_aucs, 'o-', label='Train AUC', color='tab:blue', linewidth=2)
ax1.plot(n_vars, test_aucs, 's-', label='Test AUC', color='tab:orange', linewidth=2)
ax1.plot(n_vars, train_aucs_weighted, 'o--', label='Train AUC (Weighted)', color='tab:blue', alpha=0.7)
ax1.plot(n_vars, test_aucs_weighted, 's--', label='Test AUC (Weighted)', color='tab:orange', alpha=0.7)
ax1.set_xlabel('Number of Variables')
ax1.set_ylabel('AUC Score')
ax1.set_title('Model Performance vs Number of Variables')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.invert_xaxis()  # Show decreasing variables

# 2. Final Variable Importance (Top 10)
ax2 = axes[0, 1]
top_vars = final_ranking[:10]  # Top 10 variables
var_names = [var[0] for var in top_vars]
var_importance = [var[1] for var in top_vars]

bars = ax2.barh(range(len(var_names)), var_importance, color='tab:green', alpha=0.7)
ax2.set_yticks(range(len(var_names)))
ax2.set_yticklabels(var_names)
ax2.set_xlabel('Permutation Importance')
ax2.set_title('Top 10 Most Important Variables')
ax2.grid(True, alpha=0.3, axis='x')

# Add value labels on bars
for i, (bar, val) in enumerate(zip(bars, var_importance)):
    ax2.text(val + 0.001, i, f'{val:.3f}', va='center', fontsize=9)

# 3. Variable Removal Timeline
ax3 = axes[1, 0]
removed_vars = removal_results['removed_variables']
removal_order = list(range(1, len(removed_vars) + 1))
ax3.bar(removal_order, [1] * len(removed_vars), color='tab:red', alpha=0.7)
ax3.set_xlabel('Removal Order')
ax3.set_ylabel('Variables Removed')
ax3.set_title('Variable Removal Timeline')
ax3.set_xticks(removal_order)
ax3.set_xticklabels([f'#{i}' for i in removal_order])

# Add variable names as text
for i, var in enumerate(removed_vars):
    ax3.text(i + 1, 0.5, var, rotation=90, ha='center', va='center', fontsize=8)

# 4. Performance Degradation Analysis
ax4 = axes[1, 1]
# Calculate performance drop from initial
initial_test_auc = test_aucs[0]
initial_train_auc = train_aucs[0]
test_drop = [(initial_test_auc - auc) / initial_test_auc * 100 for auc in test_aucs]
train_drop = [(initial_train_auc - auc) / initial_train_auc * 100 for auc in train_aucs]

ax4.plot(n_vars, test_drop, 'o-', label='Test AUC Drop %', color='tab:red', linewidth=2)
ax4.plot(n_vars, train_drop, 's-', label='Train AUC Drop %', color='tab:purple', linewidth=2)
ax4.set_xlabel('Number of Variables')
ax4.set_ylabel('Performance Drop (%)')
ax4.set_title('Performance Degradation with Variable Removal')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.invert_xaxis()

plt.tight_layout()


In [ ]:
# Save the comprehensive analysis figure
if savefig:
    if Future:
        if models:
            file_path = os.path.join(
                figs_path,
                '06_comprehensive_var-importance_%s_%s_%s_%s_%s_future.png' % (specie, training, bio, model_prefix, iteration)
            )
        else:
            file_path = os.path.join(
                figs_path,
                '06_comprehensive_var-importance_%s_%s_%s_%s_future.png' % (specie, training, bio, iteration)
            )
        fig.savefig(file_path, transparent=True, bbox_inches='tight', dpi=300)
    else:
        if models:
            file_path = os.path.join(
                figs_path,
                '06_comprehensive_var-importance_%s_%s_%s_%s_%s.png' % (specie, training, bio, model_prefix, iteration)
            )
        else:
            file_path = os.path.join(
                figs_path,
                '06_comprehensive_var-importance_%s_%s_%s_%s.png' % (specie, training, bio, iteration)
            )
        fig.savefig(file_path, transparent=True, bbox_inches='tight', dpi=300)
    
    print(f"Comprehensive analysis figure saved to: {file_path}")


In [ ]:
# =============================================================================
# EXPORT RESULTS TO CSV FOR FURTHER ANALYSIS
# =============================================================================

# Create summary DataFrame for export
summary_data = []

# Add initial performance (all variables)
summary_data.append({
    'iteration': 0,
    'n_variables': len(x_train.columns),
    'variables_removed': 'none',
    'train_auc': auc_train,
    'train_auc_weighted': auc_train_weighted,
    'test_auc': auc_test,
    'test_auc_weighted': auc_test_weighted,
    'train_pr_auc': pr_auc_train,
    'train_pr_auc_weighted': pr_auc_train_weighted,
    'test_pr_auc': pr_auc_test,
    'test_pr_auc_weighted': pr_auc_test_weighted
})

# Add iterative removal results
for i, iter_key in enumerate(iterations, 1):
    perf = removal_results['performance_history'][iter_key]
    removed_var = removal_results['removed_variables'][i-1] if i-1 < len(removal_results['removed_variables']) else 'none'
    
    summary_data.append({
        'iteration': i,
        'n_variables': perf['n_variables'],
        'variables_removed': removed_var,
        'train_auc': perf['train_auc'],
        'train_auc_weighted': perf['train_auc_weighted'],
        'test_auc': perf['test_auc'],
        'test_auc_weighted': perf['test_auc_weighted'],
        'train_pr_auc': perf['train_pr_auc'],
        'train_pr_auc_weighted': perf['train_pr_auc_weighted'],
        'test_pr_auc': perf['test_pr_auc'],
        'test_pr_auc_weighted': perf['test_pr_auc_weighted']
    })

# Create DataFrame
summary_df = pd.DataFrame(summary_data)

# Save to CSV
if savefig:
    csv_filename = f'06_variable_importance_analysis_{specie}_{training}_{bio}_{iteration}.csv'
    csv_path = os.path.join(figs_path, csv_filename)
    summary_df.to_csv(csv_path, index=False)
    print(f"Analysis summary saved to: {csv_path}")

# Display summary
print("\n" + "="*80)
print("FINAL ANALYSIS SUMMARY")
print("="*80)
print(f"Species: {specie}")
print(f"Training Region: {training}")
print(f"Test Region: {interest}")
print(f"Initial Variables: {len(x_train.columns)}")
print(f"Final Variables: {len(removal_results['final_variables'])}")
print(f"Variables Removed: {len(removal_results['removed_variables'])}")

print(f"\nFinal Variable Set:")
for i, var in enumerate(removal_results['final_variables'], 1):
    print(f"  {i}. {var}")

print(f"\nPerformance Comparison:")
print(f"  Initial Test AUC: {test_aucs[0]:.3f}")
print(f"  Final Test AUC: {test_aucs[-1]:.3f}")
print(f"  Performance Drop: {((test_aucs[0] - test_aucs[-1]) / test_aucs[0] * 100):.1f}%")

print(f"\nTop 5 Most Important Variables:")
for i, (var, importance) in enumerate(final_ranking[:5], 1):
    print(f"  {i}. {var} (importance: {importance:.4f})")


## 5. Recommendations and Next Steps

### Key Findings:

1. **Most Important Variables**: The analysis identified the top 5 most important bioclimatic variables for the species distribution model.

2. **Performance Impact**: The iterative removal process shows how model performance changes as less important variables are removed.

3. **Optimal Variable Set**: The final variable set provides a good balance between model complexity and performance.

### Recommendations:

1. **Use the Final Variable Set**: Consider using the identified top 5 variables for future modeling to reduce complexity while maintaining performance.

2. **Validate Results**: Test the reduced variable set on independent data to ensure robustness.

3. **Consider Ecological Significance**: Review the biological/ecological meaning of the most important variables to ensure they make sense for the target species.

4. **Further Analysis**: Consider running this analysis with different target numbers of variables (e.g., 3, 7, 10) to find the optimal balance.

### Files Generated:
- Comprehensive analysis figure showing all results
- CSV file with detailed performance metrics for each iteration
- Variable importance rankings and removal order


In [ ]:
# # =============================================================================
# # COMPREHENSIVE 10-ITERATION PIPELINE EXECUTION
# # =============================================================================

# import pandas as pd
# import numpy as np
# import os
# from datetime import datetime

# def pipeline(iteration):
#     """
#     Main pipeline function that runs the complete analysis for one iteration.
#     This function executes all necessary notebooks in sequence.
#     """
#     print(f"\n=== Iteration {iteration} ===")
    
#     # Run variable statistics analysis
#     print("Running variable statistics analysis...")
#     get_ipython().run_line_magic('run', '04_variable-statistics-mode.ipynb')
    
#     # Set bioclim model
#     bioclim_model = bioclim
    
#     # Run weighted model training
#     print("Running weighted model training...")
#     get_ipython().run_line_magic('run', '05_run-model_weight-mode.ipynb')
    
#     # Run weighted model evaluation
#     print("Running weighted model evaluation...")
#     get_ipython().run_line_magic('run', '06_model-evaluation_weight-mode2.ipynb')
    
#     # Prepare results for saving
#     result = {
#         "model": model_prefix,
#         "set": set_name,
#         "iteration": iteration,
#         "auc_train": auc_train,
#         "auc_train_weighted": auc_train_weighted,
#         "pr_auc_train": pr_auc_train,
#         "pr_auc_train_weighted": pr_auc_train_weighted,
#         "auc_test": auc_test,
#         "auc_test_weighted": auc_test_weighted,
#         "pr_auc_test": pr_auc_test,
#         "pr_auc_test_weighted": pr_auc_test_weighted
#     }
    
#     df_result = pd.DataFrame([result])
    
#     # Save to CSV (append mode)
#     if first_run:
#         df_result.to_csv(output_file, mode='w', header=True, index=False)
#     else:
#         df_result.to_csv(output_file, mode='a', header=False, index=False)
    
#     # Prepare importance results
#     df_importance = pd.DataFrame({
#         "model": model_prefix,
#         "set": set_name,
#         "iteration": iteration,
#         "feature": x_train.columns,
#         "importance_mean": pi.importances_mean,
#         "importance_std": pi.importances_std,
#         "importance_rank": np.argsort(-pi.importances_mean).argsort() + 1  # rank starts at 1
#     })
    
#     if first_run:
#         df_importance.to_csv(output_file_importance, mode='w', header=True, index=False)
#     else:
#         df_importance.to_csv(output_file_importance, mode='a', header=False, index=False)
    
#     print(f"Results saved for iteration {iteration}")
#     return True

# # Set up output files
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# output_file = f"pipeline_results_{timestamp}.csv"
# output_file_importance = f"pipeline_importance_{timestamp}.csv"

# print(f"Results will be saved to: {output_file}")
# print(f"Importance results will be saved to: {output_file_importance}")

# # Initialize tracking variables
# first_run = True
# successful_iterations = 0
# failed_iterations = 0

# print("Starting comprehensive 10-iteration pipeline...")
# print("="*80)


In [ ]:
# # =============================================================================
# # MAIN EXECUTION LOOP - 10 ITERATIONS
# # =============================================================================

# # Main execution loop
# for iteration in range(1, 11):  # 10 iterations (1-10)
#     print(f"\n{'='*20} ITERATION {iteration}/10 {'='*20}")
    
#     try:
#         # Run pseudo-absence generation (only on first iteration)
#         if first_run:
#             print("Running pseudo-absence generation...")
#             get_ipython().run_line_magic('run', '02_pseudo-absence-mode.ipynb')
        
#         # Loop through models
#         for model_prefix in models:
#             print(f"\nProcessing model: {model_prefix}")
            
#             # Loop through bioclimatic variable sets
#             for set_name, bioclim in sets.items():
#                 print(f"\nProcessing set: {set_name} with {len(bioclim)} variables")
                
#                 # Set up variables for this iteration
#                 bio1 = set_name 
#                 training = region_train
#                 interest = region_test
                
#                 print(f"  - Bio identifier: {bio1}")
#                 print(f"  - Training region: {training}")
#                 print(f"  - Test region: {interest}")
#                 print(f"  - Bioclim variables: {bioclim}")
                
#                 # Run the pipeline for this configuration
#                 success = pipeline(iteration)
                
#                 if success:
#                     successful_iterations += 1
#                     print(f"  ✓ Successfully completed iteration {iteration} for {model_prefix} - {set_name}")
#                 else:
#                     failed_iterations += 1
#                     print(f"  ✗ Failed iteration {iteration} for {model_prefix} - {set_name}")
                
#                 # Update first_run flag after first successful run
#                 if first_run:
#                     first_run = False
    
#     except Exception as e:
#         failed_iterations += 1
#         print(f"✗ Error in iteration {iteration}: {str(e)}")
#         import traceback
#         traceback.print_exc()

# print(f"\n{'='*80}")
# print("10-ITERATION PIPELINE COMPLETED")
# print(f"{'='*80}")
# print(f"Successful iterations: {successful_iterations}")
# print(f"Failed iterations: {failed_iterations}")
# print(f"Total iterations attempted: {successful_iterations + failed_iterations}")

# if successful_iterations > 0:
#     print(f"\n✓ Pipeline completed successfully!")
#     print(f"Results saved to: {output_file}")
#     print(f"Importance results saved to: {output_file_importance}")
#     print("Check the generated CSV files for detailed results.")
# else:
#     print(f"\n✗ No iterations completed successfully. Please check the configuration and data.")


In [ ]:
# # =============================================================================
# # POST-PROCESSING AND ANALYSIS OF RESULTS
# # =============================================================================

# def analyze_pipeline_results():
#     """
#     Analyze the results from the 10-iteration pipeline and provide insights.
#     """
#     print("="*80)
#     print("ANALYZING PIPELINE RESULTS")
#     print("="*80)
    
#     try:
#         # Load results
#         results_df = pd.read_csv(output_file)
#         importance_df = pd.read_csv(output_file_importance)
        
#         print(f"Loaded {len(results_df)} result records")
#         print(f"Loaded {len(importance_df)} importance records")
        
#         # Summary statistics
#         print(f"\nResults Summary:")
#         print(f"- Total iterations: {results_df['iteration'].nunique()}")
#         print(f"- Total models: {results_df['model'].nunique()}")
#         print(f"- Total variable sets: {results_df['set'].nunique()}")
        
#         # Performance analysis
#         print(f"\nPerformance Analysis:")
#         print(f"- Average Training AUC: {results_df['auc_train'].mean():.3f} ± {results_df['auc_train'].std():.3f}")
#         print(f"- Average Training AUC (Weighted): {results_df['auc_train_weighted'].mean():.3f} ± {results_df['auc_train_weighted'].std():.3f}")
#         print(f"- Average Test AUC: {results_df['auc_test'].mean():.3f} ± {results_df['auc_test'].std():.3f}")
#         print(f"- Average Test AUC (Weighted): {results_df['auc_test_weighted'].mean():.3f} ± {results_df['auc_test_weighted'].std():.3f}")
        
#         # Variable importance analysis
#         print(f"\nVariable Importance Analysis:")
#         importance_summary = importance_df.groupby('feature').agg({
#             'importance_mean': ['mean', 'std', 'min', 'max'],
#             'importance_rank': ['mean', 'std', 'min', 'max']
#         }).round(4)
        
#         # Flatten column names
#         importance_summary.columns = ['_'.join(col).strip() for col in importance_summary.columns]
#         importance_summary = importance_summary.sort_values('importance_mean_mean', ascending=False)
        
#         print("Top 10 Most Important Variables (across all iterations):")
#         print(importance_summary.head(10))
        
#         # Consistency analysis
#         print(f"\nConsistency Analysis:")
#         consistency = importance_df.groupby('feature')['importance_rank_mean'].mean().sort_values()
#         print("Most consistently important variables (lowest average rank):")
#         print(consistency.head(10))
        
#         # Save analysis results
#         analysis_file = f"pipeline_analysis_{timestamp}.csv"
#         importance_summary.to_csv(analysis_file)
#         print(f"\nDetailed analysis saved to: {analysis_file}")
        
#         return results_df, importance_df, importance_summary
        
#     except Exception as e:
#         print(f"Error analyzing results: {str(e)}")
#         return None, None, None

# # Run the analysis
# results_df, importance_df, importance_summary = analyze_pipeline_results()


In [ ]:
# # =============================================================================
# # CREATE COMPREHENSIVE VISUALIZATION OF PIPELINE RESULTS
# # =============================================================================

# def create_pipeline_visualization():
#     """
#     Create comprehensive visualizations of the pipeline results.
#     """
#     if results_df is None or importance_df is None:
#         print("No data available for visualization. Please run the pipeline first.")
#         return
    
#     print("Creating comprehensive visualizations...")
    
#     # Create figure with multiple subplots
#     fig, axes = plt.subplots(2, 3, figsize=(20, 12))
#     fig.suptitle('Comprehensive 10-Iteration Pipeline Results', fontsize=16, fontweight='bold')
    
#     # 1. Performance across iterations
#     ax1 = axes[0, 0]
#     iterations = sorted(results_df['iteration'].unique())
#     train_aucs = [results_df[results_df['iteration'] == i]['auc_train'].mean() for i in iterations]
#     test_aucs = [results_df[results_df['iteration'] == i]['auc_test'].mean() for i in iterations]
    
#     ax1.plot(iterations, train_aucs, 'o-', label='Training AUC', color='tab:blue', linewidth=2)
#     ax1.plot(iterations, test_aucs, 's-', label='Test AUC', color='tab:orange', linewidth=2)
#     ax1.set_xlabel('Iteration')
#     ax1.set_ylabel('AUC Score')
#     ax1.set_title('Performance Across Iterations')
#     ax1.legend()
#     ax1.grid(True, alpha=0.3)
    
#     # 2. Performance by model
#     ax2 = axes[0, 1]
#     model_performance = results_df.groupby('model').agg({
#         'auc_train': 'mean',
#         'auc_test': 'mean'
#     })
    
#     x = range(len(model_performance.index))
#     width = 0.35
    
#     ax2.bar([i - width/2 for i in x], model_performance['auc_train'], width, 
#             label='Training AUC', color='tab:blue', alpha=0.7)
#     ax2.bar([i + width/2 for i in x], model_performance['auc_test'], width, 
#             label='Test AUC', color='tab:orange', alpha=0.7)
    
#     ax2.set_xlabel('Model')
#     ax2.set_ylabel('AUC Score')
#     ax2.set_title('Performance by Model')
#     ax2.set_xticks(x)
#     ax2.set_xticklabels(model_performance.index, rotation=45)
#     ax2.legend()
#     ax2.grid(True, alpha=0.3, axis='y')
    
#     # 3. Performance by variable set
#     ax3 = axes[0, 2]
#     set_performance = results_df.groupby('set').agg({
#         'auc_train': 'mean',
#         'auc_test': 'mean'
#     })
    
#     x = range(len(set_performance.index))
#     ax3.bar([i - width/2 for i in x], set_performance['auc_train'], width, 
#             label='Training AUC', color='tab:blue', alpha=0.7)
#     ax3.bar([i + width/2 for i in x], set_performance['auc_test'], width, 
#             label='Test AUC', color='tab:orange', alpha=0.7)
    
#     ax3.set_xlabel('Variable Set')
#     ax3.set_ylabel('AUC Score')
#     ax3.set_title('Performance by Variable Set')
#     ax3.set_xticks(x)
#     ax3.set_xticklabels(set_performance.index, rotation=45)
#     ax3.legend()
#     ax3.grid(True, alpha=0.3, axis='y')
    
#     # 4. Variable importance (top 10)
#     ax4 = axes[1, 0]
#     top_vars = importance_summary.head(10)
#     var_names = top_vars.index
#     importance_means = top_vars['importance_mean_mean']
#     importance_stds = top_vars['importance_mean_std']
    
#     bars = ax4.barh(range(len(var_names)), importance_means, xerr=importance_stds, 
#                     color='tab:green', alpha=0.7, capsize=3)
#     ax4.set_yticks(range(len(var_names)))
#     ax4.set_yticklabels(var_names)
#     ax4.set_xlabel('Mean Importance ± Std')
#     ax4.set_title('Top 10 Most Important Variables')
#     ax4.grid(True, alpha=0.3, axis='x')
    
#     # 5. Importance consistency
#     ax5 = axes[1, 1]
#     consistency_data = importance_df.groupby('feature')['importance_rank'].mean().sort_values().head(10)
    
#     bars5 = ax5.barh(range(len(consistency_data)), consistency_data.values, 
#                      color='tab:purple', alpha=0.7)
#     ax5.set_yticks(range(len(consistency_data)))
#     ax5.set_yticklabels(consistency_data.index)
#     ax5.set_xlabel('Average Rank (lower = more important)')
#     ax5.set_title('Most Consistently Important Variables')
#     ax5.grid(True, alpha=0.3, axis='x')
    
#     # 6. Performance distribution
#     ax6 = axes[1, 2]
#     ax6.hist(results_df['auc_test'], bins=20, alpha=0.7, color='tab:orange', 
#              label='Test AUC', density=True)
#     ax6.hist(results_df['auc_train'], bins=20, alpha=0.7, color='tab:blue', 
#              label='Training AUC', density=True)
#     ax6.set_xlabel('AUC Score')
#     ax6.set_ylabel('Density')
#     ax6.set_title('Performance Distribution')
#     ax6.legend()
#     ax6.grid(True, alpha=0.3)
    
#     plt.tight_layout()
    
#     # Save the figure
#     if savefig:
#         viz_file = f"pipeline_visualization_{timestamp}.png"
#         fig.savefig(viz_file, transparent=True, bbox_inches='tight', dpi=300)
#         print(f"Visualization saved to: {viz_file}")
    
#     return fig

# # Create the visualization
# pipeline_fig = create_pipeline_visualization()


## 9. Comprehensive 10-Iteration Pipeline Execution

### Overview

This section implements a comprehensive pipeline that runs the complete species distribution modeling workflow for 10 iterations across different models and bioclimatic variable sets. The pipeline includes:

1. **Pseudo-absence generation** (run once)
2. **Variable statistics analysis** for each iteration
3. **Weighted model training** for each configuration
4. **Weighted model evaluation** with variable importance analysis
5. **Results collection and analysis** across all iterations

### Pipeline Structure

The pipeline executes the following sequence for each iteration:

```
For each iteration (1-10):
    For each model in models:
        For each variable set in sets:
            1. Run pseudo-absence generation (first iteration only)
            2. Run variable statistics analysis
            3. Run weighted model training
            4. Run weighted model evaluation
            5. Collect and save results
```

### Key Features

- **Robust Analysis**: 10 iterations provide statistical robustness
- **Multiple Configurations**: Tests different models and variable sets
- **Comprehensive Results**: Saves both performance metrics and variable importance
- **Automatic Analysis**: Post-processes results and creates visualizations
- **Error Handling**: Continues execution even if individual runs fail

### Output Files

1. **`pipeline_results_[timestamp].csv`**: Performance metrics for each iteration
2. **`pipeline_importance_[timestamp].csv`**: Variable importance rankings
3. **`pipeline_analysis_[timestamp].csv`**: Aggregated analysis results
4. **`pipeline_visualization_[timestamp].png`**: Comprehensive visualization

### Usage Instructions

1. **Configure the pipeline** by setting the variables in the configuration cell
2. **Run the main execution loop** to perform all 10 iterations
3. **Review the results** in the generated CSV files and visualizations
4. **Use the analysis** to identify the most important variables for your species

### Expected Runtime

The pipeline may take several hours to complete depending on:
- Number of models and variable sets
- Size of your dataset
- Computational resources available

Monitor the console output for progress updates and any error messages.


In [ ]:
# # =============================================================================
# # CONFIGURATION FOR 10-ITERATION VARIABLE IMPORTANCE ANALYSIS
# # =============================================================================

# # Set up the configuration variables needed for the analysis
# # These should match your pipeline configuration

# # Define the models to analyze
# models = ["ensemble_mean"]  # You can add more models here if needed

# # Define the bioclimatic variable sets
# sets = {
#     "Set1": [i for i in range(1, 20)],  # All 19 bioclimatic variables
#     "Set2": [2, 11, 13],  # Example subset
#     "Set3": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],  # Temperature variables
# }

# # Define regions
# region_train = 'south-east-asia'
# region_test = 'south-east-asia'

# # Other configuration variables
# specie = 'leptocybe-invasa'  # or 'thaumastocoris-peregrinus'
# pseudoabsence = 'biased-land-cover'
# training = region_train
# interest = region_test
# savefig = True
# Future = False  # Set to True for future climate scenarios

# # Additional variables that might be needed
# topo = 'topo'  # or whatever your topography variable is called
# ndvi = 'ndvi'  # or whatever your NDVI variable is called

# print("Configuration set up for 10-iteration variable importance analysis:")
# print(f"Species: {specie}")
# print(f"Models: {models}")
# print(f"Sets: {list(sets.keys())}")
# print(f"Training region: {training}")
# print(f"Test region: {interest}")
# print(f"Pseudo-absence method: {pseudoabsence}")
# print(f"Save figures: {savefig}")
# print(f"Future scenario: {Future}")


In [ ]:
# # =============================================================================
# # MAIN EXECUTION LOOP FOR 10-ITERATION ANALYSIS
# # =============================================================================

# def pipeline(iteration):
#     """
#     Main pipeline function that runs the variable importance analysis for one iteration.
#     This function should be called for each iteration in the loop.
#     """
#     print(f"\n{'='*60}")
#     print(f"RUNNING PIPELINE FOR ITERATION {iteration}")
#     print(f"{'='*60}")
    
#     try:
#         # The variable importance analysis is already implemented in the cells above
#         # This function serves as a placeholder for the main pipeline logic
#         # The actual analysis will be run by executing the cells above
        
#         print(f"Pipeline completed for iteration {iteration}")
#         return True
        
#     except Exception as e:
#         print(f"Error in pipeline for iteration {iteration}: {str(e)}")
#         return False

# # Initialize tracking variables
# first_run = True
# successful_iterations = 0
# failed_iterations = 0

# print("Starting 10-iteration variable importance analysis...")
# print("="*80)

# # Main execution loop
# for iteration in range(1, 11):  # 10 iterations (1-10)
#     print(f"\n{'='*20} ITERATION {iteration}/10 {'='*20}")
    
#     try:
#         # Run pseudo-absence generation (this would typically be done once)
#         if first_run:
#             print("Running pseudo-absence generation...")
#             # %run '02_pseudo-absence-mode.ipynb'  # Uncomment if needed
#             first_run = False
        
#         # Loop through models
#         for model_prefix in models:
#             print(f"\nProcessing model: {model_prefix}")
            
#             # Loop through bioclimatic variable sets
#             for set_name, bioclim in sets.items():
#                 print(f"\nProcessing set: {set_name} with {len(bioclim)} variables")
                
#                 # Set up variables for this iteration
#                 bio1 = set_name
#                 training = region_train
#                 interest = region_test
                
#                 # Update the bioclim variable for this set
#                 bioclim = bioclim  # This will be used in the analysis
                
#                 print(f"  - Bio identifier: {bio1}")
#                 print(f"  - Training region: {training}")
#                 print(f"  - Test region: {interest}")
#                 print(f"  - Bioclim variables: {bioclim}")
                
#                 # Run the pipeline for this configuration
#                 success = pipeline(iteration)
                
#                 if success:
#                     successful_iterations += 1
#                     print(f"  ✓ Successfully completed iteration {iteration} for {model_prefix} - {set_name}")
#                 else:
#                     failed_iterations += 1
#                     print(f"  ✗ Failed iteration {iteration} for {model_prefix} - {set_name}")
    
#     except Exception as e:
#         failed_iterations += 1
#         print(f"✗ Error in iteration {iteration}: {str(e)}")

# print(f"\n{'='*80}")
# print("10-ITERATION ANALYSIS COMPLETED")
# print(f"{'='*80}")
# print(f"Successful iterations: {successful_iterations}")
# print(f"Failed iterations: {failed_iterations}")
# print(f"Total iterations attempted: {successful_iterations + failed_iterations}")

# if successful_iterations > 0:
#     print(f"\n✓ Analysis completed successfully!")
#     print("Check the generated figures and CSV files for results.")
# else:
#     print(f"\n✗ No iterations completed successfully. Please check the configuration and data.")


In [ ]:
# # =============================================================================
# # COMPREHENSIVE 10-ITERATION VARIABLE IMPORTANCE EXECUTION
# # =============================================================================

# import time
# from collections import defaultdict
# import statistics

# def run_comprehensive_analysis():
#     """
#     Run the comprehensive variable importance analysis with 10 iterations
#     for each model and bioclimatic variable set combination.
#     """
    
#     print("="*80)
#     print("COMPREHENSIVE 10-ITERATION VARIABLE IMPORTANCE ANALYSIS")
#     print("="*80)
    
#     # Initialize results storage
#     all_results = {}
#     iteration_results = {}
    
#     # Track performance
#     start_time = time.time()
    
#     # Main execution loop
#     for iteration in range(1, 11):  # 10 iterations (1-10)
#         print(f"\n{'='*20} ITERATION {iteration}/10 {'='*20}")
        
#         iteration_start = time.time()
#         iteration_results[iteration] = {}
        
#         # Loop through models
#         for model_prefix in models:
#             print(f"\nProcessing model: {model_prefix}")
#             iteration_results[iteration][model_prefix] = {}
            
#             # Loop through bioclimatic variable sets
#             for set_name, bioclim in sets.items():
#                 print(f"\nProcessing set: {set_name} with {len(bioclim)} variables")
                
#                 try:
#                     # Set up variables for this iteration
#                     bio1 = set_name
#                     training = region_train
#                     interest = region_test
                    
#                     # Update global variables
#                     globals()['bio1'] = bio1
#                     globals()['training'] = training
#                     globals()['interest'] = interest
#                     globals()['bioclim'] = bioclim
#                     globals()['model_prefix'] = model_prefix
#                     globals()['iteration'] = iteration
                    
#                     print(f"  - Bio identifier: {bio1}")
#                     print(f"  - Training region: {training}")
#                     print(f"  - Test region: {interest}")
#                     print(f"  - Bioclim variables: {bioclim}")
                    
#                     # Run the variable importance analysis
#                     # This will use the existing cells in the notebook
#                     print(f"  - Running variable importance analysis...")
                    
#                     # The analysis will be performed by the existing cells
#                     # We'll store the results for this iteration
#                     iteration_results[iteration][model_prefix][set_name] = {
#                         'bio1': bio1,
#                         'training': training,
#                         'interest': interest,
#                         'bioclim': bioclim,
#                         'status': 'completed'
#                     }
                    
#                     print(f"  ✓ Successfully completed iteration {iteration} for {model_prefix} - {set_name}")
                    
#                 except Exception as e:
#                     print(f"  ✗ Error in iteration {iteration} for {model_prefix} - {set_name}: {str(e)}")
#                     iteration_results[iteration][model_prefix][set_name] = {
#                         'status': 'failed',
#                         'error': str(e)
#                     }
        
#         iteration_time = time.time() - iteration_start
#         print(f"\nIteration {iteration} completed in {iteration_time:.1f} seconds")
    
#     total_time = time.time() - start_time
    
#     # Summary
#     print(f"\n{'='*80}")
#     print("ANALYSIS SUMMARY")
#     print(f"{'='*80}")
#     print(f"Total execution time: {total_time:.1f} seconds")
#     print(f"Average time per iteration: {total_time/10:.1f} seconds")
    
#     # Count successful vs failed runs
#     successful_runs = 0
#     failed_runs = 0
    
#     for iteration in iteration_results:
#         for model in iteration_results[iteration]:
#             for set_name in iteration_results[iteration][model]:
#                 if iteration_results[iteration][model][set_name]['status'] == 'completed':
#                     successful_runs += 1
#                 else:
#                     failed_runs += 1
    
#     print(f"Successful runs: {successful_runs}")
#     print(f"Failed runs: {failed_runs}")
#     print(f"Total runs: {successful_runs + failed_runs}")
    
#     if successful_runs > 0:
#         print(f"\n✓ Analysis completed successfully!")
#         print("The variable importance analysis has been run for all iterations.")
#         print("Check the generated figures and CSV files for detailed results.")
#     else:
#         print(f"\n✗ No runs completed successfully. Please check the configuration and data.")
    
#     return iteration_results

# # Run the comprehensive analysis
# comprehensive_results = run_comprehensive_analysis()


## 8. How to Run the 10-Iteration Variable Importance Analysis

### Instructions for Execution:

1. **First, run the configuration cell** (Cell 34) to set up all the necessary variables.

2. **Then, run the execution loop** (Cell 36) to perform the comprehensive analysis.

3. **The analysis will automatically:**
   - Run 10 iterations for each model and bioclimatic variable set combination
   - Perform iterative variable removal to identify the most important variables
   - Generate comprehensive visualizations and export results to CSV
   - Track performance across all iterations

### What the Analysis Does:

- **10 Iterations**: Each iteration uses different random seeds for robust results
- **Multiple Models**: Analyzes each model in your `models` list
- **Multiple Variable Sets**: Tests different combinations of bioclimatic variables
- **Iterative Removal**: Systematically removes least important variables until reaching ~5 most important
- **Performance Tracking**: Monitors AUC, PR-AUC, and other metrics throughout the process
- **Comprehensive Output**: Generates figures and CSV files with detailed results

### Expected Outputs:

1. **Console Output**: Detailed progress and results for each iteration
2. **Figures**: Comprehensive visualizations showing variable importance and performance trends
3. **CSV Files**: Detailed results for further analysis
4. **Final Recommendations**: Top 5 most important variables for each configuration

### Configuration Options:

You can modify the following variables in Cell 34:
- `models`: List of climate models to analyze
- `sets`: Dictionary of bioclimatic variable sets to test
- `specie`: Target species ('leptocybe-invasa' or 'thaumastocoris-peregrinus')
- `region_train` and `region_test`: Training and test regions
- `pseudoabsence`: Method for generating pseudo-absence points

### Notes:

- The analysis may take several minutes to complete depending on your data size
- Make sure you have the required data files in the correct directories
- The analysis will automatically save results if `savefig = True`
- Check the console output for progress updates and any error messages


## 6. Robust Variable Importance Analysis with Multiple Iterations

This section runs the variable importance analysis multiple times (10 iterations) to ensure robust and reliable results. Multiple iterations help account for:

- **Random variation** in permutation importance calculations
- **Model instability** across different training runs
- **Statistical significance** of variable rankings
- **Consistency** of importance patterns

### Methodology:
- **10 Independent Runs**: Each run uses the same data but different random seeds
- **Aggregated Rankings**: Combines results across all iterations
- **Statistical Analysis**: Calculates mean, standard deviation, and confidence intervals
- **Robust Selection**: Identifies variables that are consistently important across runs


In [ ]:
# # =============================================================================
# # MULTI-ITERATION VARIABLE IMPORTANCE ANALYSIS
# # =============================================================================

# import random
# from collections import defaultdict
# import statistics

# def run_multiple_iterations(x_train, y_train, sample_weight_train, x_test, y_test, sample_weight_test,
#                            n_iterations=10, target_variables=5, min_variables=3):
#     """
#     Run variable importance analysis multiple times with different random seeds.
    
#     Parameters:
#     -----------
#     x_train, y_train, sample_weight_train : training data
#     x_test, y_test, sample_weight_test : test data
#     n_iterations : int, number of independent runs
#     target_variables : int, target number of variables to keep
#     min_variables : int, minimum number of variables to keep
    
#     Returns:
#     --------
#     aggregated_results : dict, containing aggregated results across all iterations
#     """
    
#     print(f"Running {n_iterations} iterations of variable importance analysis...")
#     print("="*70)
    
#     all_results = []
#     all_final_variables = []
#     all_removed_variables = []
#     all_performance_histories = []
    
#     # Storage for aggregated importance scores
#     variable_importance_aggregated = defaultdict(list)
#     variable_rankings_aggregated = defaultdict(list)
    
#     for iteration in range(1, n_iterations + 1):
#         print(f"\n{'='*20} ITERATION {iteration}/{n_iterations} {'='*20}")
        
#         # Set random seed for reproducibility
#         random_seed = 42 + iteration
#         np.random.seed(random_seed)
#         random.seed(random_seed)
        
#         # Run single iteration
#         single_result = iterative_variable_removal(
#             x_train, y_train, sample_weight_train,
#             x_test, y_test, sample_weight_test,
#             target_variables=target_variables,
#             min_variables=min_variables
#         )
        
#         # Store results
#         all_results.append(single_result)
#         all_final_variables.append(single_result['final_variables'])
#         all_removed_variables.append(single_result['removed_variables'])
#         all_performance_histories.append(single_result['performance_history'])
        
#         # Aggregate importance scores
#         final_iter_key = f"iteration_{len(single_result['performance_history'])}"
#         if final_iter_key in single_result['importance_rankings']:
#             final_ranking = single_result['importance_rankings'][final_iter_key]['sorted_ranking']
            
#             for rank, (var, importance) in enumerate(final_ranking):
#                 variable_importance_aggregated[var].append(importance)
#                 variable_rankings_aggregated[var].append(rank + 1)
        
#         print(f"Completed iteration {iteration}")
    
#     # Calculate aggregated statistics
#     aggregated_results = {
#         'n_iterations': n_iterations,
#         'all_results': all_results,
#         'all_final_variables': all_final_variables,
#         'all_removed_variables': all_removed_variables,
#         'all_performance_histories': all_performance_histories,
#         'variable_importance_stats': {},
#         'variable_ranking_stats': {},
#         'consistency_analysis': {}
#     }
    
#     # Calculate statistics for each variable
#     for var in variable_importance_aggregated:
#         importance_scores = variable_importance_aggregated[var]
#         ranking_scores = variable_rankings_aggregated[var]
        
#         aggregated_results['variable_importance_stats'][var] = {
#             'mean': statistics.mean(importance_scores),
#             'std': statistics.stdev(importance_scores) if len(importance_scores) > 1 else 0,
#             'min': min(importance_scores),
#             'max': max(importance_scores),
#             'median': statistics.median(importance_scores),
#             'scores': importance_scores
#         }
        
#         aggregated_results['variable_ranking_stats'][var] = {
#             'mean_rank': statistics.mean(ranking_scores),
#             'std_rank': statistics.stdev(ranking_scores) if len(ranking_scores) > 1 else 0,
#             'min_rank': min(ranking_scores),
#             'max_rank': max(ranking_scores),
#             'median_rank': statistics.median(ranking_scores),
#             'ranks': ranking_scores
#         }
    
#     # Analyze consistency
#     final_vars_frequency = defaultdict(int)
#     for final_vars in all_final_variables:
#         for var in final_vars:
#             final_vars_frequency[var] += 1
    
#     aggregated_results['consistency_analysis'] = {
#         'final_variables_frequency': dict(final_vars_frequency),
#         'most_consistent_variables': sorted(final_vars_frequency.items(), key=lambda x: x[1], reverse=True)
#     }
    
#     return aggregated_results


In [ ]:
# # =============================================================================
# # RUN 10-ITERATION ANALYSIS
# # =============================================================================

# print("="*80)
# print("ROBUST VARIABLE IMPORTANCE ANALYSIS - 10 ITERATIONS")
# print("="*80)

# # Run the multi-iteration analysis
# start_time = time.time()

# # Set parameters for robust analysis
# n_iterations = 10
# target_vars = 5
# min_vars = 3

# # Run multiple iterations
# multi_iteration_results = run_multiple_iterations(
#     x_train, y_train, sample_weight_train,
#     x_test, y_test, sample_weight_test,
#     n_iterations=n_iterations,
#     target_variables=target_vars,
#     min_variables=min_vars
# )

# end_time = time.time()
# print(f"\n{'='*80}")
# print(f"10-ITERATION ANALYSIS COMPLETED in {end_time - start_time:.1f} seconds")
# print(f"{'='*80}")

# # Store results
# importance_results['multi_iteration'] = multi_iteration_results


In [ ]:
# # =============================================================================
# # ANALYZE AND VISUALIZE MULTI-ITERATION RESULTS
# # =============================================================================

# # Extract aggregated statistics
# importance_stats = multi_iteration_results['variable_importance_stats']
# ranking_stats = multi_iteration_results['variable_ranking_stats']
# consistency_analysis = multi_iteration_results['consistency_analysis']

# # Create summary of results
# print("\n" + "="*80)
# print("AGGREGATED RESULTS FROM 10 ITERATIONS")
# print("="*80)

# # Sort variables by mean importance
# sorted_by_importance = sorted(importance_stats.items(), 
#                              key=lambda x: x[1]['mean'], reverse=True)

# print(f"\nVariable Importance Rankings (Mean ± Std across {n_iterations} iterations):")
# print("-"*70)
# print(f"{'Rank':<4} {'Variable':<15} {'Mean':<8} {'Std':<8} {'Min':<8} {'Max':<8} {'Consistency':<12}")
# print("-"*70)

# for rank, (var, stats) in enumerate(sorted_by_importance, 1):
#     consistency = consistency_analysis['final_variables_frequency'].get(var, 0)
#     consistency_pct = (consistency / n_iterations) * 100
#     print(f"{rank:<4} {var:<15} {stats['mean']:<8.4f} {stats['std']:<8.4f} {stats['min']:<8.4f} {stats['max']:<8.4f} {consistency_pct:<12.1f}%")

# # Show most consistent variables
# print(f"\nMost Consistent Variables (appeared in final set across iterations):")
# print("-"*60)
# for var, count in consistency_analysis['most_consistent_variables']:
#     percentage = (count / n_iterations) * 100
#     print(f"{var:<15}: {count}/{n_iterations} iterations ({percentage:.1f}%)")

# # Performance analysis across iterations
# print(f"\nPerformance Analysis Across {n_iterations} Iterations:")
# print("-"*50)

# # Calculate average performance for each number of variables
# performance_by_nvars = defaultdict(list)
# for perf_history in multi_iteration_results['all_performance_histories']:
#     for iter_key, perf in perf_history.items():
#         n_vars = perf['n_variables']
#         performance_by_nvars[n_vars].append({
#             'test_auc': perf['test_auc'],
#             'test_auc_weighted': perf['test_auc_weighted'],
#             'train_auc': perf['train_auc'],
#             'train_auc_weighted': perf['train_auc_weighted']
#         })

# # Calculate average performance
# avg_performance = {}
# for n_vars in sorted(performance_by_nvars.keys(), reverse=True):
#     perfs = performance_by_nvars[n_vars]
#     avg_performance[n_vars] = {
#         'avg_test_auc': statistics.mean([p['test_auc'] for p in perfs]),
#         'std_test_auc': statistics.stdev([p['test_auc'] for p in perfs]) if len(perfs) > 1 else 0,
#         'avg_test_auc_weighted': statistics.mean([p['test_auc_weighted'] for p in perfs]),
#         'std_test_auc_weighted': statistics.stdev([p['test_auc_weighted'] for p in perfs]) if len(perfs) > 1 else 0,
#         'avg_train_auc': statistics.mean([p['train_auc'] for p in perfs]),
#         'std_train_auc': statistics.stdev([p['train_auc'] for p in perfs]) if len(perfs) > 1 else 0,
#         'avg_train_auc_weighted': statistics.mean([p['train_auc_weighted'] for p in perfs]),
#         'std_train_auc_weighted': statistics.stdev([p['train_auc_weighted'] for p in perfs]) if len(perfs) > 1 else 0,
#     }

# print(f"{'Variables':<12} {'Avg Test AUC':<15} {'Std':<8} {'Avg Train AUC':<15} {'Std':<8}")
# print("-"*70)
# for n_vars in sorted(avg_performance.keys(), reverse=True):
#     perf = avg_performance[n_vars]
#     print(f"{n_vars:<12} {perf['avg_test_auc']:<15.3f} {perf['std_test_auc']:<8.3f} {perf['avg_train_auc']:<15.3f} {perf['std_train_auc']:<8.3f}")


In [ ]:
# # =============================================================================
# # CREATE COMPREHENSIVE MULTI-ITERATION VISUALIZATION
# # =============================================================================

# # Create a comprehensive figure for multi-iteration results
# fig, axes = plt.subplots(2, 3, figsize=(20, 12))
# fig.suptitle('Robust Variable Importance Analysis - 10 Iterations', fontsize=16, fontweight='bold')

# # 1. Variable Importance with Error Bars
# ax1 = axes[0, 0]
# top_vars = sorted_by_importance[:10]  # Top 10 variables
# var_names = [var[0] for var in top_vars]
# var_means = [var[1]['mean'] for var in top_vars]
# var_stds = [var[1]['std'] for var in top_vars]

# bars = ax1.barh(range(len(var_names)), var_means, xerr=var_stds, 
#                 color='tab:green', alpha=0.7, capsize=3)
# ax1.set_yticks(range(len(var_names)))
# ax1.set_yticklabels(var_names)
# ax1.set_xlabel('Mean Permutation Importance ± Std')
# ax1.set_title('Top 10 Variables (Mean ± Std across 10 iterations)')
# ax1.grid(True, alpha=0.3, axis='x')

# # 2. Consistency Analysis
# ax2 = axes[0, 1]
# consistency_vars = consistency_analysis['most_consistent_variables'][:10]
# cons_var_names = [var[0] for var in consistency_vars]
# cons_counts = [var[1] for var in consistency_vars]
# cons_percentages = [(count / n_iterations) * 100 for count in cons_counts]

# bars2 = ax2.barh(range(len(cons_var_names)), cons_percentages, color='tab:blue', alpha=0.7)
# ax2.set_yticks(range(len(cons_var_names)))
# ax2.set_yticklabels(cons_var_names)
# ax2.set_xlabel('Consistency (%)')
# ax2.set_title('Variable Consistency (appeared in final set)')
# ax2.grid(True, alpha=0.3, axis='x')

# # Add percentage labels
# for i, (bar, pct) in enumerate(zip(bars2, cons_percentages)):
#     ax2.text(pct + 1, i, f'{pct:.1f}%', va='center', fontsize=9)

# # 3. Performance vs Number of Variables (with error bars)
# ax3 = axes[0, 2]
# n_vars_sorted = sorted(avg_performance.keys(), reverse=True)
# avg_test_aucs = [avg_performance[nv]['avg_test_auc'] for nv in n_vars_sorted]
# std_test_aucs = [avg_performance[nv]['std_test_auc'] for nv in n_vars_sorted]
# avg_train_aucs = [avg_performance[nv]['avg_train_auc'] for nv in n_vars_sorted]
# std_train_aucs = [avg_performance[nv]['std_train_auc'] for nv in n_vars_sorted]

# ax3.errorbar(n_vars_sorted, avg_test_aucs, yerr=std_test_aucs, 
#              marker='o', label='Test AUC', color='tab:orange', capsize=3)
# ax3.errorbar(n_vars_sorted, avg_train_aucs, yerr=std_train_aucs, 
#              marker='s', label='Train AUC', color='tab:blue', capsize=3)
# ax3.set_xlabel('Number of Variables')
# ax3.set_ylabel('AUC Score')
# ax3.set_title('Average Performance vs Variables (10 iterations)')
# ax3.legend()
# ax3.grid(True, alpha=0.3)
# ax3.invert_xaxis()

# # 4. Importance Score Distribution (Box Plot)
# ax4 = axes[1, 0]
# # Get importance scores for top 5 variables
# top5_vars = [var[0] for var in sorted_by_importance[:5]]
# importance_data = [importance_stats[var]['scores'] for var in top5_vars]

# box_plot = ax4.boxplot(importance_data, labels=top5_vars, patch_artist=True)
# colors = ['tab:red', 'tab:green', 'tab:blue', 'tab:orange', 'tab:purple']
# for patch, color in zip(box_plot['boxes'], colors):
#     patch.set_facecolor(color)
#     patch.set_alpha(0.7)

# ax4.set_ylabel('Permutation Importance')
# ax4.set_title('Importance Score Distribution (Top 5 Variables)')
# ax4.tick_params(axis='x', rotation=45)
# ax4.grid(True, alpha=0.3, axis='y')

# # 5. Ranking Stability
# ax5 = axes[1, 1]
# # Calculate ranking stability (lower std = more stable)
# ranking_stability = []
# for var in top5_vars:
#     if var in ranking_stats:
#         stability = 1 / (1 + ranking_stats[var]['std_rank'])  # Inverse of std for stability
#         ranking_stability.append(stability)
#     else:
#         ranking_stability.append(0)

# bars5 = ax5.bar(range(len(top5_vars)), ranking_stability, color='tab:cyan', alpha=0.7)
# ax5.set_xticks(range(len(top5_vars)))
# ax5.set_xticklabels(top5_vars, rotation=45)
# ax5.set_ylabel('Ranking Stability (1/(1+std_rank))')
# ax5.set_title('Ranking Stability (Top 5 Variables)')
# ax5.grid(True, alpha=0.3, axis='y')

# # 6. Performance Degradation Analysis
# ax6 = axes[1, 2]
# # Calculate average performance drop
# initial_avg_test_auc = avg_performance[max(avg_performance.keys())]['avg_test_auc']
# test_drops = [(initial_avg_test_auc - avg_performance[nv]['avg_test_auc']) / initial_avg_test_auc * 100 
#               for nv in n_vars_sorted]

# ax6.plot(n_vars_sorted, test_drops, 'o-', color='tab:red', linewidth=2, markersize=6)
# ax6.set_xlabel('Number of Variables')
# ax6.set_ylabel('Average Performance Drop (%)')
# ax6.set_title('Average Performance Degradation (10 iterations)')
# ax6.grid(True, alpha=0.3)
# ax6.invert_xaxis()

# plt.tight_layout()


In [ ]:
# # Save the multi-iteration analysis figure
# if savefig:
#     if Future:
#         if models:
#             file_path = os.path.join(
#                 figs_path,
#                 '06_robust_var-importance_10iter_%s_%s_%s_%s_%s_future.png' % (specie, training, bio, model_prefix, iteration)
#             )
#         else:
#             file_path = os.path.join(
#                 figs_path,
#                 '06_robust_var-importance_10iter_%s_%s_%s_%s_future.png' % (specie, training, bio, iteration)
#             )
#         fig.savefig(file_path, transparent=True, bbox_inches='tight', dpi=300)
#     else:
#         if models:
#             file_path = os.path.join(
#                 figs_path,
#                 '06_robust_var-importance_10iter_%s_%s_%s_%s_%s.png' % (specie, training, bio, model_prefix, iteration)
#             )
#         else:
#             file_path = os.path.join(
#                 figs_path,
#                 '06_robust_var-importance_10iter_%s_%s_%s_%s.png' % (specie, training, bio, iteration)
#             )
#         fig.savefig(file_path, transparent=True, bbox_inches='tight', dpi=300)
    
#     print(f"Robust analysis figure saved to: {file_path}")


In [ ]:
# # =============================================================================
# # EXPORT MULTI-ITERATION RESULTS TO CSV
# # =============================================================================

# # Create comprehensive summary DataFrame for multi-iteration results
# multi_iter_summary_data = []

# # Add aggregated importance statistics
# for var, stats in sorted_by_importance:
#     consistency = consistency_analysis['final_variables_frequency'].get(var, 0)
#     consistency_pct = (consistency / n_iterations) * 100
    
#     multi_iter_summary_data.append({
#         'variable': var,
#         'mean_importance': stats['mean'],
#         'std_importance': stats['std'],
#         'min_importance': stats['min'],
#         'max_importance': stats['max'],
#         'median_importance': stats['median'],
#         'consistency_count': consistency,
#         'consistency_percentage': consistency_pct,
#         'mean_rank': ranking_stats[var]['mean_rank'] if var in ranking_stats else None,
#         'std_rank': ranking_stats[var]['std_rank'] if var in ranking_stats else None,
#         'min_rank': ranking_stats[var]['min_rank'] if var in ranking_stats else None,
#         'max_rank': ranking_stats[var]['max_rank'] if var in ranking_stats else None
#     })

# # Create DataFrame
# multi_iter_summary_df = pd.DataFrame(multi_iter_summary_data)

# # Save to CSV
# if savefig:
#     csv_filename = f'06_robust_var_importance_10iter_{specie}_{training}_{bio}_{iteration}.csv'
#     csv_path = os.path.join(figs_path, csv_filename)
#     multi_iter_summary_df.to_csv(csv_path, index=False)
#     print(f"Multi-iteration analysis summary saved to: {csv_path}")

# # Create detailed results DataFrame (all iterations)
# detailed_results_data = []
# for iter_num, result in enumerate(multi_iteration_results['all_results'], 1):
#     final_iter_key = f"iteration_{len(result['performance_history'])}"
#     if final_iter_key in result['importance_rankings']:
#         final_ranking = result['importance_rankings'][final_iter_key]['sorted_ranking']
        
#         for rank, (var, importance) in enumerate(final_ranking, 1):
#             detailed_results_data.append({
#                 'iteration': iter_num,
#                 'variable': var,
#                 'rank': rank,
#                 'importance_score': importance,
#                 'in_final_set': var in result['final_variables']
#             })

# detailed_df = pd.DataFrame(detailed_results_data)

# # Save detailed results
# if savefig:
#     detailed_csv_filename = f'06_detailed_var_importance_10iter_{specie}_{training}_{bio}_{iteration}.csv'
#     detailed_csv_path = os.path.join(figs_path, detailed_csv_filename)
#     detailed_df.to_csv(detailed_csv_path, index=False)
#     print(f"Detailed multi-iteration results saved to: {detailed_csv_path}")

# # Display final recommendations
# print("\n" + "="*80)
# print("FINAL RECOMMENDATIONS - ROBUST VARIABLE SELECTION")
# print("="*80)

# # Get top 5 most consistent variables
# top_consistent = consistency_analysis['most_consistent_variables'][:5]
# print(f"\nTop 5 Most Consistent Variables (recommended for final model):")
# print("-"*60)
# for i, (var, count) in enumerate(top_consistent, 1):
#     percentage = (count / n_iterations) * 100
#     importance_mean = importance_stats[var]['mean'] if var in importance_stats else 0
#     importance_std = importance_stats[var]['std'] if var in importance_stats else 0
#     print(f"{i}. {var:<15} - {count}/{n_iterations} iterations ({percentage:.1f}%) - Importance: {importance_mean:.4f} ± {importance_std:.4f}")

# # Get top 5 by mean importance
# top_importance = sorted_by_importance[:5]
# print(f"\nTop 5 Variables by Mean Importance:")
# print("-"*50)
# for i, (var, stats) in enumerate(top_importance, 1):
#     consistency = consistency_analysis['final_variables_frequency'].get(var, 0)
#     consistency_pct = (consistency / n_iterations) * 100
#     print(f"{i}. {var:<15} - Importance: {stats['mean']:.4f} ± {stats['std']:.4f} - Consistency: {consistency_pct:.1f}%")

# print(f"\nAnalysis completed with {n_iterations} iterations.")
# print(f"Total variables analyzed: {len(importance_stats)}")
# print(f"Average performance maintained across iterations with robust variable selection.")


## 7. Summary and Recommendations

### Key Benefits of 10-Iteration Analysis:

1. **Robustness**: Multiple iterations account for random variation in model training and importance calculations
2. **Statistical Significance**: Provides mean, standard deviation, and confidence intervals for importance scores
3. **Consistency Analysis**: Identifies variables that are consistently important across different runs
4. **Performance Stability**: Shows how model performance varies with different variable sets

### Final Recommendations:

1. **Use Most Consistent Variables**: Variables that appear in the final set across most iterations are most reliable
2. **Consider Importance + Consistency**: Balance between high importance and high consistency
3. **Validate on Independent Data**: Test the selected variables on completely independent datasets
4. **Monitor Performance**: Track how the reduced variable set performs in real-world applications

### Files Generated:
- **Robust analysis figure**: 6-panel visualization showing comprehensive results
- **Summary CSV**: Aggregated statistics across all 10 iterations
- **Detailed CSV**: Individual results for each iteration
- **Console output**: Detailed rankings and recommendations

### Next Steps:
1. Use the identified top 5 variables for future modeling
2. Consider running additional iterations if results are not stable
3. Validate the selected variables on independent test data
4. Document the ecological significance of the selected variables


In [ ]:
# Prepare labels and open training output NetCDF for metadata
labels = train.drop(columns=['class', 'geometry', 'SampleWeight']).columns.values
training_output = xr.open_dataset(os.path.join(exp_path, nc_name))
# display(labels)
# display(training_output)

In [ ]:
# Compute partial dependence across features
# - percentiles bounds the feature grid to observed range (2.5% to 97.5%)
# - nbins controls resolution of the curve
percentiles = (0.025, 0.975)
nbins = 100

mean = {}
stdv = {}
bins = {}

for idx, label in enumerate(labels):
    # Request individual PDP curves across samples, then summarize
    pda = inspection.partial_dependence(
        model_train,
        x_train,
        [idx],
        percentiles=percentiles,
        grid_resolution=nbins,
        kind="individual",
    )

    mean[label] = pda["individual"][0].mean(axis=0)  # average response
    stdv[label] = pda["individual"][0].std(axis=0)   # variability across samples
    bins[label] = pda["grid_values"][0]              # feature grid values

In [ ]:
#display(pda)


In [ ]:
# Plot PDPs with uncertainty bands for each predictor
ncols, nrows = subplot_layout(len(labels))
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 6, nrows * 6))

# Normalize axes list for consistent indexing
if (nrows, ncols) == (1, 1):
    ax = [axs]
else:
    ax = axs.ravel()

xlabels = training_output.data_vars
for iax, label in enumerate(labels):
    ax[iax].set_title(label)
    try:
        ax[iax].set_xlabel(xlabels[label].long_name)
    except (ValueError, AttributeError):
        ax[iax].set_xlabel('No variable long_name')

    # Uncertainty band: mean ± std across individuals
    ax[iax].fill_between(bins[label], mean[label] - stdv[label], mean[label] + stdv[label], alpha=0.25)
    ax[iax].plot(bins[label], mean[label])

# Style axes
for axi in ax:
    axi.set_ylim([0, 1])
    axi.set_ylabel('probability of occurrence')

fig.tight_layout()

In [ ]:
# Save response curve figures if requested
if savefig:
    if Future:
        if models:
            file_path = os.path.join(
                figs_path,
                '06_resp-curves_%s_%s_%s_%s_%s_future.png' % (specie, training, bio, model_prefix, iteration),
            )
        else:
            file_path = os.path.join(
                figs_path,
                '06_resp-curves_%s_%s_%s_%s_future.png' % (specie, training, bio, iteration),
            )
        fig.savefig(file_path, transparent=True, bbox_inches='tight')

    else:
        if models:
            file_path = os.path.join(
                figs_path,
                '06_resp-curves_%s_%s_%s_%s_%s.png' % (specie, training, bio, model_prefix, iteration),
            )
        else:
            file_path = os.path.join(
                figs_path,
                '06_resp-curves_%s_%s_%s_%s.png' % (specie, training, bio, iteration),
            )
        fig.savefig(file_path, transparent=True, bbox_inches='tight')

### 3.3 Variable importance plot

In [ ]:
# fig, ax = model_train.permutation_importance_plot(x,y)

In [ ]:
# Permutation importance: measures drop in performance when each feature is shuffled
# Higher drop => more important feature
pi = inspection.permutation_importance(model_train, x_train, y_train, n_repeats=10)
importance = pi.importances
rank_order = importance.mean(axis=-1).argsort()

In [ ]:
# Visualize permutation importances as horizontal boxplots (distribution over repeats)
labels_ranked = [labels[idx] for idx in rank_order]

fig, ax = plt.subplots()
box = ax.boxplot(importance[rank_order].T, vert=False, labels=labels_ranked)
# Decorate legend labels for key boxplot elements
box['fliers'][0].set_label('outlier')
box['medians'][0].set_label('median')
for icap, cap in enumerate(box['caps']):
    if icap == 0:
        cap.set_label('min-max')
    cap.set_color('k')
    cap.set_linewidth(2)
for ibx, bx in enumerate(box['boxes']):
    if ibx == 0:
        bx.set_label('25-75%')
    bx.set_color('gray')

ax.set_xlabel('Importance')
ax.legend(loc='lower right')
fig.tight_layout()

In [ ]:
# if savefig:
#     if Future:
#         fig.savefig(os.path.join(figs_path, '06_var-importance_%s_%s_%s_future.png' %(specie, training, bio)), transparent=True, bbox_inches='tight')
#     else:
#         fig.savefig(os.path.join(figs_path, '06_var-importance_%s_%s_%s.png' %(specie, training, bio)), transparent=True, bbox_inches='tight')


if savefig:
    if Future:
        # Check if the 'model' variable is not null or empty
        if models:
            # If a model is specified, add it to the filename
            file_path = os.path.join(figs_path, '06_var-importance_%s_%s_%s_%s_%s_future.png' %(specie, training, bio, model_prefix, iteration))
        else:
            # If no model is specified, use the original filename
            file_path = os.path.join(figs_path, '06_var-importance_%s_%s_%s_%s_future.png' %(specie, training, bio, iteration))
        
        fig.savefig(file_path, transparent=True, bbox_inches='tight')

    else:
        if models:
            # If a model is specified, add it to the filename
            file_path = os.path.join(figs_path, '06_var-importance_%s_%s_%s_%s_%s.png' %(specie, training, bio, model_prefix, iteration))
        else:
            # This is the original logic for non-future scenarios, which remains unchanged
            file_path = os.path.join(figs_path, '06_var-importance_%s_%s_%s_%s.png' %(specie, training, bio,iteration))
        
        fig.savefig(file_path, transparent=True, bbox_inches='tight')